# Análise de Dados de Ocorrências Aeronáuticas - CENIPA

## Desafio de Ciência de Dados Di2win

Este notebook documenta o processo completo de análise dos dados de ocorrências aeronáuticas, desde a inspeção inicial dos dados brutos até a modelagem e conclusões.

### 0. Instalação das Bibliotecas

Esta célula inicial garante que todas as bibliotecas necessárias para a análise estejam instaladas no ambiente.

In [ ]:
# O '-q' é para uma instalação "quieta" (quiet), sem exibir muitas mensagens
!pip install -q pandas matplotlib seaborn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Inspeção Inicial dos Dados

O primeiro passo fundamental é carregar cada um dos arquivos `.csv` em um DataFrame do Pandas e realizar uma inspeção básica para entender sua estrutura. Para cada arquivo, vamos verificar:

1.  `.head()`: Para visualizar as 5 primeiras linhas e ter uma noção do conteúdo.
2.  `.info()`: Para entender os tipos de dados de cada coluna e a contagem de valores não-nulos.
3.  `.shape`: Para saber o número exato de linhas e colunas.

In [1]:
import pandas as pd

# --- OCORRÊNCIA ---
print("="*40)
print("INSPECIONANDO O ARQUIVO: ocorrencia.csv")
print("="*40)
df_ocorrencia = pd.read_csv('data/ocorrencia.csv', sep=';', encoding='latin1', low_memory=False)
print("\nFormato do DataFrame (linhas, colunas):", df_ocorrencia.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_ocorrencia.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_ocorrencia.head())

INSPECIONANDO O ARQUIVO: ocorrencia.csv

Formato do DataFrame (linhas, colunas): (13185, 22)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13185 entries, 0 to 13184
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               13185 non-null  int64 
 1   codigo_ocorrencia1              13185 non-null  int64 
 2   codigo_ocorrencia2              13185 non-null  int64 
 3   codigo_ocorrencia3              13185 non-null  int64 
 4   codigo_ocorrencia4              13185 non-null  int64 
 5   ocorrencia_classificacao        13185 non-null  object
 6   ocorrencia_latitude             10425 non-null  object
 7   ocorrencia_longitude            10425 non-null  object
 8   ocorrencia_cidade               13185 non-null  object
 9   ocorrencia_uf                   13185 non-null  object
 10  ocorrencia_pais               

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,87125,87125,87125,87125,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,...,11/05/2025,04:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,87124,87124,87124,87124,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,...,08/05/2025,14:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2,87123,87123,87123,87123,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,...,09/05/2025,18:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
3,87122,87122,87122,87122,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,...,04/05/2025,14:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
4,87121,87121,87121,87121,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,...,10/05/2025,10:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO


### 1.1. Primeiras Observações sobre `ocorrencia.csv`

Com base na inspeção inicial, levantamos os seguintes pontos cruciais sobre este arquivo:

* **Chaves e Colunas Repetidas:** Existem 5 colunas de código (`codigo_ocorrencia` a `codigo_ocorrencia4`). Nossa hipótese é que elas servem como "chaves estrangeiras" para conectar este `DataFrame` com os outros (aeronave, recomendação, etc.). Precisamos verificar se os valores nelas são idênticos.
* **Tipos de Dados:** Colunas de data e hora (`ocorrencia_dia`, `ocorrencia_hora`) estão como texto (`object`). Para qualquer análise temporal, será necessário convertê-las para o tipo `datetime`.
* **Dados Ausentes (Nulos):** Há uma quantidade significativa de valores nulos, especialmente em:
    * `ocorrencia_latitude` / `ocorrencia_longitude`
    * `divulgacao_relatorio_numero`
    * `divulgacao_dia_publicacao` (a grande maioria é nula)
* **Valores Sentinela:** Uma inspeção visual no `.head()` revela o uso de `***` em `divulgacao_relatorio_numero`. Este é um "valor sentinela" usado para representar dados não aplicáveis ou não preenchidos. Precisamos tratar isso como um valor nulo padrão (`NaN`) para que o Pandas possa lidar com ele corretamente.

Nossa próxima etapa será focar em validar essas hipóteses e iniciar a limpeza e o pré-processamento dos dados.

In [2]:
print("--- Investigando e Limpando as Colunas de Código em 'df_ocorrencia' ---")

# 1. Verificando se os valores nas 5 colunas de código são idênticos para cada linha
colunas_codigo = ['codigo_ocorrencia', 'codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']
sao_identicas = (df_ocorrencia[colunas_codigo].nunique(axis=1) == 1).all()


# 2. Agindo com base na verificação
if sao_identicas:
    print("\n[CONFIRMADO] A hipótese é verdadeira. Os valores nas 5 colunas de código são idênticos para todas as linhas.")
    
    # Se são idênticas, podemos remover as 4 colunas redundantes com segurança
    colunas_para_remover = ['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']
    df_ocorrencia_limpo = df_ocorrencia.drop(columns=colunas_para_remover)
    
    print(f"\nColunas removidas: {colunas_para_remover}")
    print("\nO DataFrame 'df_ocorrencia' agora possui apenas a chave 'codigo_ocorrencia'.")
    
else:
    print("\n[FALHA] A hipótese é falsa. Existem linhas onde os valores das colunas de código são diferentes.")
    # Se não forem idênticas, não fazemos nada por enquanto e mantemos o dataframe original
    df_ocorrencia_limpo = df_ocorrencia.copy()


# --- Resultados ---
print("\nVisualizando a estrutura do novo DataFrame 'df_ocorrencia_limpo':")
display(df_ocorrencia_limpo.head())
print(f"\nNovo formato do DataFrame (linhas, colunas): {df_ocorrencia_limpo.shape}")

--- Investigando e Limpando as Colunas de Código em 'df_ocorrencia' ---

[CONFIRMADO] A hipótese é verdadeira. Os valores nas 5 colunas de código são idênticos para todas as linhas.

Colunas removidas: ['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']

O DataFrame 'df_ocorrencia' agora possui apenas a chave 'codigo_ocorrencia'.

Visualizando a estrutura do novo DataFrame 'df_ocorrencia_limpo':


,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,BRASIL,FAER,11/05/2025,04:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,BRASIL,SBUL,08/05/2025,14:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,BRASIL,SBGR,09/05/2025,18:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
3,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,BRASIL,SBSM,04/05/2025,14:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
4,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,BRASIL,SBSR,10/05/2025,10:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO



Novo formato do DataFrame (linhas, colunas): (13185, 18)


### 1.2. Limpeza de Tipos de Dados e Tratamento de Valores Sentinela

Com a chave primária corrigida, o próximo passo da limpeza é tratar duas outras questões importantes que identificamos no `df_ocorrencia_limpo`:

1.  **Conversão de Data e Hora:** As colunas `ocorrencia_dia` e `ocorrencia_hora` estão como texto (`object`). Vamos convertê-las para o tipo `datetime`, que é o formato correto para manipulação de datas e horas, e uni-las em uma única coluna `ocorrencia_data_hora` para facilitar análises temporais.
2.  **Substituição de Valores Sentinela:** Vamos percorrer o `DataFrame` para substituir todos os marcadores de texto como `***` e outros formatos não-padrão por um valor nulo (`NaN`), que o Pandas entende como "ausente". Isso nos dará uma visão precisa da quantidade real de dados faltantes.

In [3]:
print("--- Limpando Tipos de Dados e Valores Sentinela em 'df_ocorrencia_limpo' ---")

# 1. Tratamento de data e hora
# O errors='coerce' garante que, se houver um formato inválido, ele se tornará nulo (NaT)
df_ocorrencia_limpo['ocorrencia_dia'] = pd.to_datetime(df_ocorrencia_limpo['ocorrencia_dia'], errors='coerce')

# Para a hora, também usamos 'coerce'. Alguns valores podem ser inválidos.
df_ocorrencia_limpo['ocorrencia_hora'] = pd.to_timedelta(df_ocorrencia_limpo['ocorrencia_hora'], errors='coerce')

# Combinando data e hora em uma única coluna. Ignoramos os que tiverem data ou hora nula.
df_ocorrencia_limpo['ocorrencia_data_hora'] = df_ocorrencia_limpo['ocorrencia_dia'] + df_ocorrencia_limpo['ocorrencia_hora']

print("Coluna 'ocorrencia_data_hora' criada e colunas de data/hora originais convertidas.")


# 2. Substituindo valores sentinela e tratando inconsistências
# Vamos varrer todas as colunas do tipo 'object' (texto)
for col in df_ocorrencia_limpo.select_dtypes(include=['object']).columns:
    # Remove espaços em branco no início/fim e substitui vários marcadores por Nulos
    df_ocorrencia_limpo[col] = df_ocorrencia_limpo[col].str.strip()
    df_ocorrencia_limpo[col] = df_ocorrencia_limpo[col].replace(['***', '****', '*****', 'NULL', 'NÃO IDENTIFICADA'], pd.NA, regex=False)

print("Valores sentinela ('***', etc.) substituídos por valores nulos.")


# 3. Verificando a situação final dos tipos e dos nulos
print("\nInformações do DataFrame após a limpeza:")
df_ocorrencia_limpo.info()

print("\nContagem de valores nulos por coluna (após limpeza de sentinelas):")
display(df_ocorrencia_limpo.isnull().sum())

--- Limpando Tipos de Dados e Valores Sentinela em 'df_ocorrencia_limpo' ---
Coluna 'ocorrencia_data_hora' criada e colunas de data/hora originais convertidas.
Valores sentinela ('***', etc.) substituídos por valores nulos.

Informações do DataFrame após a limpeza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13185 entries, 0 to 13184
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype          
---  ------                          --------------  -----          
 0   codigo_ocorrencia               13185 non-null  int64          
 1   ocorrencia_classificacao        13185 non-null  object         
 2   ocorrencia_latitude             9161 non-null   object         
 3   ocorrencia_longitude            9160 non-null   object         
 4   ocorrencia_cidade               13127 non-null  object         
 5   ocorrencia_uf                   13164 non-null  object         
 6   ocorrencia_pais                 13185 non-null  object         
 7  

codigo_ocorrencia                     0
ocorrencia_classificacao              0
ocorrencia_latitude                4024
ocorrencia_longitude               4025
ocorrencia_cidade                    58
ocorrencia_uf                        21
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                     7855
ocorrencia_hora                       2
investigacao_aeronave_liberada        0
investigacao_status                   1
divulgacao_relatorio_numero        9890
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao         10677
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_data_hora               7856
dtype: int64

In [4]:
df_ocorrencia_limpo.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_data_hora
0,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,BRASIL,FAER,2025-11-05,0 days 04:20:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-11-05 04:20:00
1,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,BRASIL,SBUL,2025-08-05,0 days 14:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-08-05 14:00:00
2,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,BRASIL,SBGR,2025-09-05,0 days 18:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-09-05 18:45:00
3,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,BRASIL,SBSM,2025-04-05,0 days 14:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-04-05 14:45:00
4,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,BRASIL,SBSR,2025-10-05,0 days 10:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-10-05 10:00:00


### 1.3. Análise e Estratégia para Dados Ausentes

A limpeza de tipos e de valores sentinela nos revelou a real quantidade de dados faltantes em cada coluna. Agora, precisamos de uma estratégia para lidar com eles.

Nossa análise da contagem de nulos mostra os seguintes pontos críticos:

* **Localização (`ocorrencia_latitude`, `ocorrencia_longitude`):** Possuem cerca de 4.000 valores nulos. É inviável "inventar" ou preencher esses dados.
    * **Estratégia:** Para análises que dependem de mapas ou geolocalização, teremos que trabalhar com o subconjunto de dados que possui essa informação. Para outras análises, podemos ignorar essas colunas.

* **Data da Ocorrência (`ocorrencia_dia`, `ocorrencia_data_hora`):** A conversão para `datetime` resultou em mais de 7.800 valores nulos. Isso é um **ponto de atenção gravíssimo**. Significa que mais da metade das datas no formato original não puderam ser convertidas. Precisaremos investigar o formato dessas datas inválidas.
    * **Estratégia Imediata:** Qualquer análise temporal ficará restrita aos ~5.300 registros com data válida.

* **Dados da Divulgação (`divulgacao_relatorio_numero`, `divulgacao_dia_publicacao`):** Com ~9.900 e ~10.700 nulos respectivamente, essas colunas têm pouquíssima informação. A hipótese é que a maioria das ocorrências (especialmente incidentes) não gera um relatório público.
    * **Estratégia:** Para o nosso objetivo de analisar as *causas* das ocorrências, essas colunas sobre a *divulgação do relatório* têm pouca utilidade. Removê-las irá simplificar muito nosso `DataFrame` sem grande perda de informação para a nossa análise.

* **Local (`ocorrencia_cidade`, `ocorrencia_uf`):** Possuem uma quantidade muito pequena de nulos (58 e 21).
    * **Estratégia:** O impacto de remover essas poucas linhas do nosso conjunto de dados de mais de 13.000 registros é mínimo. A forma mais simples e limpa de tratar é remover as linhas onde a cidade ou o UF são nulos.

Com base nisso, nosso próximo passo de código será aplicar essa limpeza.

In [5]:
print("--- Aplicando Estratégia para Dados Ausentes ---")

# Vamos continuar trabalhando com o df_ocorrencia_limpo
df_ocorrencia_tratado = df_ocorrencia_limpo.copy()


# 1. Removendo colunas com alta cardinalidade de nulos e pouca relevância para a análise de causas
colunas_para_remover = ['divulgacao_relatorio_numero', 'divulgacao_dia_publicacao']
df_ocorrencia_tratado.drop(columns=colunas_para_remover, inplace=True)

print(f"Colunas removidas: {colunas_para_remover}")


# 2. Removendo linhas onde a cidade ou o UF são nulos (impacto mínimo)
linhas_antes = df_ocorrencia_tratado.shape[0]
df_ocorrencia_tratado.dropna(subset=['ocorrencia_cidade', 'ocorrencia_uf'], inplace=True)
linhas_depois = df_ocorrencia_tratado.shape[0]

print(f"\nLinhas removidas por falta de cidade/UF: {linhas_antes - linhas_depois}")


# --- Resultados ---
print("\nInformações do DataFrame após o tratamento de nulos:")
df_ocorrencia_tratado.info()

--- Aplicando Estratégia para Dados Ausentes ---
Colunas removidas: ['divulgacao_relatorio_numero', 'divulgacao_dia_publicacao']

Linhas removidas por falta de cidade/UF: 60

Informações do DataFrame após o tratamento de nulos:
<class 'pandas.core.frame.DataFrame'>
Index: 13125 entries, 0 to 13184
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype          
---  ------                          --------------  -----          
 0   codigo_ocorrencia               13125 non-null  int64          
 1   ocorrencia_classificacao        13125 non-null  object         
 2   ocorrencia_latitude             9141 non-null   object         
 3   ocorrencia_longitude            9140 non-null   object         
 4   ocorrencia_cidade               13125 non-null  object         
 5   ocorrencia_uf                   13125 non-null  object         
 6   ocorrencia_pais                 13125 non-null  object         
 7   ocorrencia_aerodromo            13125 non-

In [6]:
# --- OCORRÊNCIA TIPO ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: ocorrencia_tipo.csv")
print("="*40)
df_ocorrencia_tipo = pd.read_csv('data/ocorrencia_tipo.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_ocorrencia_tipo.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_ocorrencia_tipo.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_ocorrencia_tipo.head())



INSPECIONANDO O ARQUIVO: ocorrencia_tipo.csv

Formato do DataFrame (linhas, colunas): (13900, 4)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13900 entries, 0 to 13899
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_ocorrencia1         13900 non-null  int64 
 1   ocorrencia_tipo            13895 non-null  object
 2   ocorrencia_tipo_categoria  13895 non-null  object
 3   taxonomia_tipo_icao        13895 non-null  object
dtypes: int64(1), object(3)
memory usage: 434.5+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,87125,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,87124,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,87123,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,87122,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
4,87121,COLISÃO COM AVE,COLISÃO COM AVE,BIRD


In [7]:
# --- AERONAVE ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: aeronave.csv")
print("="*40)
df_aeronave = pd.read_csv('data/aeronave.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_aeronave.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_aeronave.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_aeronave.head())



INSPECIONANDO O ARQUIVO: aeronave.csv

Formato do DataFrame (linhas, colunas): (13301, 23)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13301 entries, 0 to 13300
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia2           13301 non-null  int64  
 1   aeronave_matricula           13300 non-null  object 
 2   aeronave_operador_categoria  13292 non-null  object 
 3   aeronave_tipo_veiculo        13300 non-null  object 
 4   aeronave_fabricante          13296 non-null  object 
 5   aeronave_modelo              13291 non-null  object 
 6   aeronave_tipo_icao           13290 non-null  object 
 7   aeronave_motor_tipo          12828 non-null  object 
 8   aeronave_motor_quantidade    13300 non-null  object 
 9   aeronave_pmd                 13300 non-null  float64
 10  aeronave_pmd_categoria       13300 non-null  float64
 

,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,87125,PSAEX,***,AVIÃO,EMBRAER,ERJ 190-400,E295,NaN,***,61200.0,61200.0,0.0,2024.0,BRASIL,BRASIL,AVIÃO,***,VIRACOPOS,ORLANDO BEZERRA DE MENEZES,DESCIDA,REGULAR,NENHUM,0.0
1,87124,PRYXD,***,AVIÃO,ATR - GIE AVIONS DE TRANSPORT RÉGIONAL,ATR-72-212A (600),AT76,***,***,23000.0,23000.0,NaN,2024.0,BRASIL,BRASIL,AVIÃO,***,TENENTE-CORONEL AVIADOR CÉSAR BOMBONATO,TANCREDO NEVES,SUBIDA,REGULAR,NENHUM,0.0
2,87123,PRXBT,***,***,AIRBUS S.A.S.,A320-271N,A20N,NaN,BIMOTOR,79000.0,79000.0,NaN,NaN,BRASIL,BRASIL,***,***,GOVERNADOR ANDRÉ FRANCO MONTORO,PINTO MARTINS,DECOLAGEM,REGULAR,NENHUM,0.0
3,87122,PRAKJ,***,AVIÃO,ATR - GIE AVIONS DE TRANSPORT RÉGIONAL,ATR-72-212A (600),AT76,TURBOÉLICE,BIMOTOR,22000.0,22000.0,0.0,2016.0,BRASIL,BRASIL,AVIÃO,***,SANTA MARIA,VIRACOPOS,DECOLAGEM,REGULAR,NENHUM,0.0
4,87121,PRMHK,***,AVIÃO,AIRBUS,A320-214,A320,JATO,BIMOTOR,77000.0,77000.0,184.0,2007.0,BRASIL,BRASIL,AVIÃO,***,PROFESSOR ERIBERTO MANOEL REINO,GOVERNADOR ANDRÉ FRANCO MONTORO,DECOLAGEM,REGULAR,LEVE,0.0


In [8]:
# --- FATOR CONTRIBUINTE ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: fator_contribuinte.csv")
print("="*40)
df_fator_contribuinte = pd.read_csv('data/fator_contribuinte.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_fator_contribuinte.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_fator_contribuinte.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_fator_contribuinte.head())



INSPECIONANDO O ARQUIVO: fator_contribuinte.csv

Formato do DataFrame (linhas, colunas): (8613, 5)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8613 entries, 0 to 8612
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   codigo_ocorrencia3   8613 non-null   int64 
 1   fator_nome           8613 non-null   object
 2   fator_aspecto        8613 non-null   object
 3   fator_condicionante  8613 non-null   object
 4   fator_area           8613 non-null   object
dtypes: int64(1), object(4)
memory usage: 336.6+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,85239,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,85239,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
2,85239,CONDIÇÕES METEOROLÓGICAS ADVERSAS,ELEMENTOS RELACIONADOS AO AMBIENTE OPERACIONAL,***,FATOR OPERACIONAL
3,85239,PERCEPÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
4,85239,PROCESSO DECISÓRIO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO


In [9]:
# --- RECOMENDAÇÃO ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: recomendacao.csv")
print("="*40)
df_recomendacao = pd.read_csv('data/recomendacao.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_recomendacao.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_recomendacao.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_recomendacao.head())



INSPECIONANDO O ARQUIVO: recomendacao.csv

Formato do DataFrame (linhas, colunas): (3388, 9)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3388 entries, 0 to 3387
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   codigo_ocorrencia4               3387 non-null   object
 1   recomendacao_numero              2890 non-null   object
 2   recomendacao_dia_assinatura      2890 non-null   object
 3   recomendacao_dia_encaminhamento  2890 non-null   object
 4   recomendacao_dia_feedback        2217 non-null   object
 5   recomendacao_conteudo            2889 non-null   object
 6   recomendacao_status              2889 non-null   object
 7   recomendacao_destinatario_sigla  2889 non-null   object
 8   recomendacao_destinatario        2889 non-null   object
dtypes: object(9)
memory usage: 238.3+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,84857,A-092/CENIPA/2024 - 01,2025-04-22,2025-04-23,NaN,Divulgar os ensinamentos colhidos na presente ...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,83193,A-014/CENIPA/2024 - 01,2025-02-17,2025-02-19,NaN,Divulgar os ensinamentos colhidos na presente ...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,83193,A-014/CENIPA/2024 - 02,2025-02-17,2025-02-19,2025-03-07,Divulgar os ensinamentos colhidos na presente ...,ADOTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,82458,A-146/CENIPA/2023 - 01,2025-03-13,2025-04-08,2025-05-09,Divulgar os ensinamentos colhidos na presente ...,ADOTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,82458,A-146/CENIPA/2023 - 02,2025-03-13,2025-04-08,NaN,Avaliar a pertinência de revisar os requisitos...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
